In [4]:
import pandas as pd
import requests
import datetime as dt
import plotly.express as px


# Create a percent sign that is in the form of a string and add to the URL. This is because % signs break up strings in python
def create_maturity_data():
    per = "%"
    url = f'https://www.wsj.com/market-data/bonds/treasuries?id={per}7B{per}22treasury{per}22{per}3A{per}22NOTES_AND_BONDS{per}22{per}7D&type=mdc_treasury'
    print(url)

    # Request treasury data from WSJ, user agent is required but doesn't seem to need a specific name
    r = requests.get(url, headers={'User-Agent':'test'})


    # Convert to Json, pull out the dictionary, convert to DataFrame
    json_data = r.json()
    data_dict = json_data["data"]["instruments"]
    df = pd.DataFrame.from_dict(data_dict)


    # convert askyield into a number and then group by the maturity date to average
    df['askYield'] = df['askYield'].astype(float)
    df['askYield'] = df['askYield']/100
    df['maturityDate'] = df['maturityDate'].astype('datetime64[ns]')
    avg_yield_df = df.groupby(['maturityDate']).mean()
    avg_yield_df = avg_yield_df.sort_values(by=['maturityDate'])


    # Return the average yield
    return(avg_yield_df.reset_index())


# Create table with yearly yield data for the next 20 years
def create_yearly_table(maturity_data):
    # Calculate the number of days to today
    maturity_data['days_to_recent'] = (maturity_data['maturityDate'] - maturity_data['maturityDate'][0]).dt.days


    # Calculate the number of years until the bond matures based on 365 days per year and takes the absolute value for all bonds for x number of years for 20 years
    for i in range(1,21):
        maturity_data[f'{i}_years'] = abs(maturity_data['days_to_recent'] - (365 * i))


    # Create data frame to work with
    period_df = pd.DataFrame(columns=('maturityDate', 'askYield', 'year'))


    # Create function that finds a maturity date that is closed to a date within x years from now for the next 20 years
    for i in range(1,21):
        # For each year sort by closest date for that year
        maturity_data = maturity_data.sort_values(by=[f'{i}_years'])
        # Take this closest date and append it to the data frame
        temp_maturity_data = maturity_data[0:1][['maturityDate', 'askYield']]
        temp_maturity_data['year'] = i
        period_df = period_df.append(temp_maturity_data)


    # Calculate the total return for each bond, you will need this for a more simplified calculation later
    period_df['askYield_totalreturn'] = (1+period_df['askYield'])**period_df['year']


    # Return this information
    return period_df.reset_index(drop=True)


# Calculate the 10 year forward for the next 10 years
def create_10_year_forward(period_df):
    ten_year_forward = pd.DataFrame(columns=('forward_year', '10_year_forward'))


    # Calculate the forward rate for each year. Zero forward is slightlty different and is used in the first if statement
    for i in range (0,11):
        if i == 0:
            temp_ten_year = period_df[9:10][['year','askYield_totalreturn']]
            temp_ten_year['forward_year'] = 0
            temp_ten_year['10_year_forward'] = temp_ten_year['askYield_totalreturn']**(1/10) - 1
            ten_year_forward = ten_year_forward.append(temp_ten_year[['forward_year', '10_year_forward']])
        else:
            temp_ten_year = period_df[i:i+1][['year','askYield_totalreturn']]
            temp_ten_year['10_year_forward'] = (period_df['askYield_totalreturn'][i+9]/period_df['askYield_totalreturn'][i-1])**(1/10) - 1
            temp_ten_year['forward_year'] = i
            ten_year_forward = ten_year_forward.append(temp_ten_year[['forward_year', '10_year_forward']])
    
    # Return table consisting of the forward rates
    return ten_year_forward


# Pull Data from WSJ
maturity_data = create_maturity_data()
# Calculate and create the total return for each year
period_df = create_yearly_table(maturity_data)
# Create forward rates for 10 year treasury
ten_year_forward_table = create_10_year_forward(period_df)


# Plot the implied forward curve
ten_year_forward_table
fig = px.scatter(x=ten_year_forward_table['forward_year'], y = ten_year_forward_table['10_year_forward'], title="10 Year Treasury Forward Rate Curve")
fig.show()

https://www.wsj.com/market-data/bonds/treasuries?id=%7B%22treasury%22%3A%22NOTES_AND_BONDS%22%7D&type=mdc_treasury


C:\Users\chris\AppData\Local\Temp\ipykernel_47304\2164317179.py:27: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.

C:\Users\chris\AppData\Local\Temp\ipykernel_47304\2164317179.py:57: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\chris\AppData\Local\Temp\ipykernel_47304\2164317179.py:57: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\chris\AppData\Local\Temp\ipykernel_47304\2164317179.py:57: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\chris\AppData\Local\Temp\ipykernel_47304\2164317179.py:57: FutureWarning:

